In [ ]:
import mne
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import numpy as np
import pandas as pd

고정실,김가람,김득실,김영현,김충연,민병춘1,박주연1,박주연2,벌,안중훈,윤병시,이미우,임석봉,전창희,정광훈2,정금례,정용태,이귀임,정복연,김정한,정광훈1,조진욱


In [ ]:
# 분석대상 이름 입력
name_list = list(input("분석대상 이름을 입력하세요. (여러명일 경우 쉼표로 구분)").split(','))

# 분석 시작
for name in name_list:
    print(f"분석대상 이름은 {name}입니다.")

    # Specify the full path to EEG data file (EDF format)
    file_path = rf'H:\Mg_EEG\tfr_files\{name}_7200_tfr.h5'
    tfr = mne.time_frequency.read_tfrs(file_path)
    print(f"성공적으로 tfr 파일을 로딩하였습니다.")

    # Saving raw file for back-up
    tfr_raw = tfr.copy()
    print(f"tfr_raw에 tfr을 복사하였습니다.")

    # 플로팅할 데이터셋 생성
    tfr_mean = np.mean(tfr.data, axis=0)
    tfr_mean_mean = np.mean(tfr_mean, axis=0)
    tfr_mean_raw = tfr_mean.copy()
    tfr_mean_mean_raw = tfr_mean_mean.copy()

    # large artifact data load
    csv_file_path = r'C:\Users\esin4\OneDrive\바탕 화면\Github\Mg_infusion_coma\large_artifact.csv'

    large_artifact_data = pd.read_csv(csv_file_path, encoding='utf-8-sig')

    # 입력한 이름이 데이터프레임에 존재하는지 확인하고 좌표 가져오기
    if name in large_artifact_data['Name'].values:
        # 입력한 이름에 해당하는 데이터 가져오기
        coordinates = large_artifact_data[large_artifact_data['Name'] == name]['Coordinates'].values[0]
    
        # 좌표가 'skip'이면 빈 리스트로 설정
        if coordinates == 'skip':
            large_artifact = []
        else:
            # 좌표 문자열을 리스트로 변환
            import ast
            large_artifact = ast.literal_eval(coordinates)
    else:
        print(f"{input_name}이(가) 데이터에 없습니다.")
        large_artifact = []

    # 결과 출력
    print(f"large_artifact 리스트:")
    print(large_artifact)

    # 인덱스로 바꾸기
    bad_idx=[]

    for t_start, t_end in large_artifact:
        bad_idx.extend(range(t_start*200, t_end*200))

    # Bad index 지우는 경우에만 실행
    tfr_mean[:, bad_idx] = tfr_mean.min()
    tfr_mean_mean[bad_idx] = tfr_mean_mean.min()

    print(f"Bad index를 제거하였습니다.")

    # Plotting
    # tfr_mean_mean 계산
    tfr_mean_mean = np.mean(tfr_mean, axis=0)

    # 이동평균 계산 (윈도우 크기 설정)
    window_size = 200*60 # window size 1 minute
    tfr_mean_mean_smooth = pd.Series(tfr_mean_mean).rolling(window=window_size, center=True).mean()

    # Create the plot
    fig, ax1 = plt.subplots(figsize=(15, 3))

    # Use pcolormesh to create the time-frequency plot
    im = ax1.pcolormesh(tfr.times, tfr.freqs, tfr_mean, 
                        norm=colors.LogNorm(vmin=tfr_mean.min(), vmax=tfr_mean.max()),
                        cmap='magma', shading='auto')

    # artifact 구간 표시
    for start, end in large_artifact:
        ax1.axvspan(start, end, color='white', alpha=1.0)

    # Add labels and title for the first plot
    ax1.set_xlabel('Time (s)')
    ax1.set_ylabel('Frequency (Hz)')

    # Add a colorbar for the first plot
    cbar = fig.colorbar(im, ax=ax1)
    cbar.set_label(r'Power ($log(\mu V^2)$)')

    # Create a second y-axis for the second plot
    ax2 = ax1.twinx()

    # tfr.times에 대한 tfr_mean_mean의 이동평균 추세선 플롯 생성
    ax2.plot(tfr.times, tfr_mean_mean_smooth, label='TFR Mean (Moving Average)', color='yellow')

    # Add labels and title for the second plot
    ax2.set_ylabel(r'Mean Power ($\mu V^2$)')

    # Add a legend for the second plot
    ax2.legend(loc='upper right')

    # Draw a red line at 3600 seconds
    ax2.axvline(x=3600, color='cyan', linestyle='--', linewidth=2, label='Mg infusion')

    # Display the plot
    plt.tight_layout()
    plt.show()
    print(f'{name}의 tfr plot을 성공적으로 생성하였습니다.')

    # Save the plot
    fig.savefig(r'H:\Mg_EEG\plot_1108\{name}_7200_tfr.png', dpi=300)
    print(f'{name}의 tfr plot을 성공적으로 저장하였습니다.')

    # Close the plot
    plt.close(fig)
    print(f'{name}의 tfr plot을 성공적으로 닫았습니다.')